In [1]:
import sys
import os
sys.path.append('../mlp')
from data_providers import MNISTDataProvider, ModifyDataProvider
os.environ['MLP_DATA_DIR'] = '../data'
import numpy as np

In [2]:
from mlp.data_providers import MNISTDataProvider
from mlp.models import MultipleLayerModel
from mlp.layers import ReluLayer, AffineLayer
from mlp.errors import CrossEntropySoftmaxError
from mlp.initialisers import GlorotUniformInit, ConstantInit
from mlp.learning_rules import MomentumLearningRule
from mlp.optimisers import Optimiser
import matplotlib.pyplot as plt
import logging


In [3]:
# train original model 

# Seed a random number generator
seed = 31102016 
rng = np.random.RandomState(seed)

# Set up a logger object to print info about the training run to stdout
logger = logging.getLogger()
logger.setLevel(logging.INFO)
logger.handlers = [logging.StreamHandler()]

In [4]:
# test model 

# Probability of input being included in output in dropout layer
incl_prob = 0.5

input_dim, output_dim, hidden_dim = 784, 10, 125

# Use Glorot initialisation scheme for weights and zero biases
weights_init = GlorotUniformInit(rng=rng, gain=2.**0.5)
biases_init = ConstantInit(0.)

# Create three affine layer model with rectified linear non-linearities
# and dropout layers before every affine layer
model = MultipleLayerModel([
    AffineLayer(input_dim, hidden_dim, weights_init, biases_init), 
    ReluLayer(),
    AffineLayer(hidden_dim, output_dim, weights_init, biases_init)
])

# Multiclass classification therefore use cross-entropy + softmax error
error = CrossEntropySoftmaxError()

# Use a momentum learning rule - you could use an adaptive learning rule
# implemented for the coursework here instead
learning_rule = MomentumLearningRule(0.02, 0.9)
# Monitor classification accuracy during training
data_monitors={'acc': lambda y, t: (y.argmax(-1) == t.argmax(-1)).mean()}


In [5]:
def train(model, error, learning_rule, train_data, valid_data, data_monitors):
    optimiser = Optimiser(model, error, learning_rule, train_data, valid_data, data_monitors)
    num_epochs = 2
    stats_interval = 2
    stats, keys, run_time = optimiser.train(num_epochs=num_epochs, stats_interval=stats_interval)
    err_train = stats[len(stats) - 1, keys['error(train)']]
    err_valid = stats[len(stats) - 1, keys['error(valid)']]
    acc_train = stats[len(stats) - 1, keys['acc(train)']]
    acc_valid = stats[len(stats) - 1, keys['error(valid)']]
    return err_train, err_valid, acc_train, acc_valid
    

In [8]:
def compare():
    train_data = MNISTDataProvider('train')
    valid_data = MNISTDataProvider('valid')
    err_train, err_valid, acc_train, acc_valid = train(model, error, learning_rule, train_data, valid_data, data_monitors)
    
    inputs, targets = ModifyDataProvider().modify(0, .01, train_data.inputs, train_data.targets)
    train_data.inputs = np.array(inputs)
    train_data.targets = np.array(targets)
    err_train_comp, err_valid_comp, acc_train_comp, acc_valid_comp = train(model, error, learning_rule, train_data, valid_data, data_monitors)

    acc_valid_diff = ((acc_valid_comp - acc_valid) / float(acc_valid_comp)) * 100
    acc_train_diff = ((acc_train_comp - acc_train) / float(acc_train_comp)) * 100
    print("Training accuracy decreased by {0}%".format(round(acc_train_diff, 2)))
    print("Validation accuracy decreased by {0}%".format(round(acc_valid_diff, 2)))

    

In [9]:
compare()

Epoch Progress: 100%|██████████| 500/500 [00:00<00:00, 652.94it/s]Epoch 2: 0.8s to complete
    error(train)=6.84e-02, acc(train)=9.82e-01, error(valid)=9.28e-02, acc(valid)=9.73e-01
Experiment Progress: 100%|██████████| 2/2 [00:01<00:00,  1.12it/s]
/Users/ashemagalhaes/miniconda3/envs/mlp2/lib/python3.6/site-packages/ipykernel_launcher.py:31: RuntimeWarning: Mean of empty slice.
Epoch Progress: 100%|██████████| 500/500 [00:00<00:00, 666.22it/s]Epoch 2: 0.8s to complete
    error(train)=nan, acc(train)=nan, error(valid)=1.05e-01, acc(valid)=9.69e-01
Experiment Progress: 100%|██████████| 2/2 [00:01<00:00,  1.15it/s]

Training accuracy decreased by nan%
Validation accuracy decreased by 11.61%
